In [ ]:
# import os
# import pickle
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, Dropout
# from tensorflow.keras.optimizers import Adam
# from tqdm import tqdm  # For progress bars

# # Helper function to calculate top-3 accuracy
# def calculate_top3_accuracy(y_true, y_probs):
#     top_3_preds = np.argsort(y_probs, axis=1)[:, -3:]
#     matches = [y_true[i] in top_3_preds[i] for i in range(len(y_true))]
#     return np.mean(matches)

# # Helper function to calculate threshold accuracy
# def calculate_threshold_accuracy(y_true, y_pred):
#     return np.mean(np.abs(y_true - y_pred) <= 1)

# # Helper function to evaluate metrics
# def evaluate_metrics(y_true, y_pred, y_probs=None):
#     metrics = {
#         'accuracy': accuracy_score(y_true, y_pred),
#         'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
#         'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
#         'f1_score': f1_score(y_true, y_pred, average='weighted'),
#         'classification_report': classification_report(y_true, y_pred, zero_division=0),
#         'threshold_accuracy': calculate_threshold_accuracy(y_true, y_pred)
#     }
#     if y_probs is not None:
#         metrics['top_3_accuracy'] = calculate_top3_accuracy(y_true, y_probs)
#     return metrics

# # Helper function to load dataset
# def load_data(file_path):
#     print(f"Loading data from: {file_path}")
#     data = pd.read_csv(file_path)
#     label_columns = [col for col in data.columns if "slider" in col]
#     feature_columns = [col for col in data.columns if col not in label_columns + ["Timestamp", "participant_id"]]
#     X = data[feature_columns]
#     return X, label_columns, data

# # Function to train models per label
# def train_model_per_label(X_train, y_train, X_test, y_test, model_save_path, label_name, model_type="rf"):
#     os.makedirs(model_save_path, exist_ok=True)
    
#     if model_type == "rf":
#         model = RandomForestClassifier(random_state=42, n_jobs=-1)
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         y_probs = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
#         with open(os.path.join(model_save_path, f"random_forest_{label_name}.pkl"), "wb") as f:
#             pickle.dump(model, f)
#     else:
#         input_shape = (X_train.shape[1], 1)
#         X_train, X_test = np.expand_dims(X_train, axis=2), np.expand_dims(X_test, axis=2)
#         model = Sequential([
#             Conv1D(64, kernel_size=3, activation="relu", input_shape=input_shape),
#             Conv1D(32, kernel_size=3, activation="relu"),
#             Flatten(),
#             Dense(128, activation="relu"),
#             Dropout(0.3),
#             Dense(11, activation="softmax")
#         ])
#         model.compile(optimizer=Adam(0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#         model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
#         model.save(os.path.join(model_save_path, f"cnn_model_{label_name}.h5"))
#         y_probs = model.predict(X_test)
#         y_pred = y_probs.argmax(axis=1)
    
#     return evaluate_metrics(y_test, y_pred, y_probs)

# # Main training loop
# def train_all_models(data_dir, root_dir):
#     results = []
#     folds = sorted(set(f.split("_")[1] for f in os.listdir(data_dir) if f.endswith(".csv")))

#     for fold in tqdm(folds, desc="Processing Folds"):
#         train_file = os.path.join(data_dir, f"fold_{fold}_train.csv")
#         test_file = os.path.join(data_dir, f"fold_{fold}_test.csv")
        
#         if not os.path.exists(train_file) or not os.path.exists(test_file):
#             print(f"Skipping fold {fold}: Missing train or test data")
#             continue
        
#         X_train, label_columns, train_data = load_data(train_file)
#         X_test, _, test_data = load_data(test_file)
        
#         for label_column in label_columns:
#             y_train, y_test = train_data[label_column], test_data[label_column]
#             fold_save_dir = os.path.join(root_dir, f"fold_{fold}", "models", label_column)
#             os.makedirs(fold_save_dir, exist_ok=True)
            
#             rf_results = train_model_per_label(X_train, y_train, X_test, y_test, os.path.join(fold_save_dir, "random_forest"), label_column, "rf")
#             results.append({"Fold": fold, "Label": label_column, "Model": "Random Forest", **rf_results})

#             cnn_results = train_model_per_label(X_train, y_train, X_test, y_test, os.path.join(fold_save_dir, "cnn"), label_column, "cnn")
#             results.append({"Fold": fold, "Label": label_column, "Model": "CNN", **cnn_results})

#     results_df = pd.DataFrame(results)
#     results_df.to_csv("fold_model_performance_results.csv", index=False)
#     print("Model training results saved to fold_model_performance_results.csv")

# if __name__ == "__main__":
#     data_dir = "folds"  # Adjust as needed
#     root_dir = "folds_original_models"
#     train_all_models(data_dir, root_dir)


In [3]:
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm  # For progress bars

# Helper function to calculate top-3 accuracy
def calculate_top3_accuracy(y_true, y_probs):
    top_3_preds = np.argsort(y_probs, axis=1)[:, -3:]
    matches = [y_true[i] in top_3_preds[i] for i in range(len(y_true))]
    return np.mean(matches)

# Helper function to calculate threshold accuracy
def calculate_threshold_accuracy(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) <= 1)

# Helper function to evaluate metrics
def evaluate_metrics(y_true, y_pred, y_probs=None):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'f1_score': f1_score(y_true, y_pred, average='weighted'),
        'classification_report': classification_report(y_true, y_pred, zero_division=0),
        'threshold_accuracy': calculate_threshold_accuracy(y_true, y_pred)
    }
    if y_probs is not None:
        metrics['top_3_accuracy'] = calculate_top3_accuracy(y_true, y_probs)
    return metrics

# Helper function to load dataset
def load_data(file_path):
    print(f"Loading data from: {file_path}")
    data = pd.read_csv(file_path)
    label_columns = [col for col in data.columns if "slider" in col]
    feature_columns = [col for col in data.columns if col not in label_columns + ["Timestamp", "participant_id"]]
    X = data[feature_columns]
    return X, label_columns, data

# Function to train models per label
def train_model_per_label(X_train, y_train, X_test, y_test, model_save_path, label_name, model_type="rf"):
    os.makedirs(model_save_path, exist_ok=True)
    
    if model_type == "rf":
        model = RandomForestClassifier(random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probs = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
        with open(os.path.join(model_save_path, f"random_forest_{label_name}.pkl"), "wb") as f:
            pickle.dump(model, f)
    elif model_type == "cnn":
        input_shape = (X_train.shape[1], 1)
        X_train, X_test = np.expand_dims(X_train, axis=2), np.expand_dims(X_test, axis=2)
        model = Sequential([
            Conv1D(64, kernel_size=3, activation="relu", input_shape=input_shape),
            Conv1D(32, kernel_size=3, activation="relu"),
            Flatten(),
            Dense(128, activation="relu"),
            Dropout(0.3),
            Dense(11, activation="softmax")
        ])
        model.compile(optimizer=Adam(0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
        model.save(os.path.join(model_save_path, f"cnn_model_{label_name}.h5"))
        y_probs = model.predict(X_test)
        y_pred = y_probs.argmax(axis=1)
    else:  # LSTM Model
        input_shape = (X_train.shape[1], 1)
        X_train, X_test = np.expand_dims(X_train, axis=2), np.expand_dims(X_test, axis=2)
        model = Sequential([
            LSTM(64, return_sequences=True, input_shape=input_shape),
            LSTM(32),
            Dense(128, activation="relu"),
            Dropout(0.3),
            Dense(11, activation="softmax")
        ])
        model.compile(optimizer=Adam(0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
        model.save(os.path.join(model_save_path, f"lstm_model_{label_name}.h5"))
        y_probs = model.predict(X_test)
        y_pred = y_probs.argmax(axis=1)
    
    return evaluate_metrics(y_test, y_pred, y_probs)

# Main training loop
def train_all_models(data_dir, root_dir):
    results = []
    folds = sorted(set(f.split("_")[1] for f in os.listdir(data_dir) if f.endswith(".csv")))

    for fold in tqdm(folds, desc="Processing Folds"):
        train_file = os.path.join(data_dir, f"fold_{fold}_train.csv")
        test_file = os.path.join(data_dir, f"fold_{fold}_test.csv")
        
        if not os.path.exists(train_file) or not os.path.exists(test_file):
            print(f"Skipping fold {fold}: Missing train or test data")
            continue
        
        X_train, label_columns, train_data = load_data(train_file)
        X_test, _, test_data = load_data(test_file)
        
        for label_column in label_columns:
            y_train, y_test = train_data[label_column], test_data[label_column]
            fold_save_dir = os.path.join(root_dir, f"fold_{fold}", "models", label_column)
            os.makedirs(fold_save_dir, exist_ok=True)
            
            rf_results = train_model_per_label(X_train, y_train, X_test, y_test, os.path.join(fold_save_dir, "random_forest"), label_column, "rf")
            results.append({"Fold": fold, "Label": label_column, "Model": "Random Forest", **rf_results})

            cnn_results = train_model_per_label(X_train, y_train, X_test, y_test, os.path.join(fold_save_dir, "cnn"), label_column, "cnn")
            results.append({"Fold": fold, "Label": label_column, "Model": "CNN", **cnn_results})
            
            lstm_results = train_model_per_label(X_train, y_train, X_test, y_test, os.path.join(fold_save_dir, "lstm"), label_column, "lstm")
            results.append({"Fold": fold, "Label": label_column, "Model": "LSTM", **lstm_results})

    results_df = pd.DataFrame(results)
    results_df.to_csv("fold_model_performance_results.csv", index=False)
    print("Model training results saved to fold_model_performance_results.csv")

if __name__ == "__main__":
    data_dir = "folds"  # Adjust as needed
    root_dir = "folds_original_models"
    train_all_models(data_dir, root_dir)


Processing Folds:   0%|          | 0/5 [00:00<?, ?it/s]

Loading data from: folds/fold_1_train.csv
Loading data from: folds/fold_1_test.csv
Epoch 1/10
134/134 [==============================] - 1s 6ms/step - loss: 2036706.7500 - accuracy: 0.1834
Epoch 2/10
134/134 [==============================] - 1s 6ms/step - loss: 2.3143 - accuracy: 0.2495
Epoch 3/10
134/134 [==============================] - 1s 6ms/step - loss: 2.2604 - accuracy: 0.2551
Epoch 4/10
134/134 [==============================] - 1s 6ms/step - loss: 2.2162 - accuracy: 0.2551
Epoch 5/10
134/134 [==============================] - 1s 6ms/step - loss: 2.1812 - accuracy: 0.2549
Epoch 6/10
134/134 [==============================] - 1s 6ms/step - loss: 2.1527 - accuracy: 0.2551
Epoch 7/10
134/134 [==============================] - 1s 6ms/step - loss: 2.1304 - accuracy: 0.2551
Epoch 8/10
134/134 [==============================] - 1s 6ms/step - loss: 2.1128 - accuracy: 0.2551
Epoch 9/10
134/134 [==============================] - 1s 6ms/step - loss: 2.0989 - accuracy: 0.2551
Epoch 10/10

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
134/134 [==============================] - 19s 127ms/step - loss: 2.0958 - accuracy: 0.2422
Epoch 2/10
134/134 [==============================] - 17s 127ms/step - loss: 2.0286 - accuracy: 0.2434
Epoch 3/10
134/134 [==============================] - 15s 109ms/step - loss: 2.0051 - accuracy: 0.2504
Epoch 4/10
134/134 [==============================] - 15s 109ms/step - loss: 1.9997 - accuracy: 0.2457
Epoch 5/10
134/134 [==============================] - 16s 116ms/step - loss: 1.9839 - accuracy: 0.2530
Epoch 6/10
134/134 [==============================] - 13s 95ms/step - loss: 1.9752 - accuracy: 0.2434
Epoch 7/10
134/134 [==============================] - 13s 96ms/step - loss: 1.9684 - accuracy: 0.2422
Epoch 8/10
134/134 [==============================] - 14s 101ms/step - loss: 1.9481 - accuracy: 0.2607
Epoch 9/10
134/134 [==============================] - 16s 120ms/step - loss: 1.9341 - accuracy: 0.2645
Epoch 10/10
134/134 [==============================] - 16s 118ms/step - los

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

34/34 [==============================] - 0s 2ms/step
Epoch 1/10
134/134 [==============================] - 19s 128ms/step - loss: 2.1584 - accuracy: 0.1899
Epoch 2/10
134/134 [==============================] - 17s 129ms/step - loss: 2.1091 - accuracy: 0.1957
Epoch 3/10
134/134 [==============================] - 17s 129ms/step - loss: 2.0818 - accuracy: 0.2164
Epoch 4/10
134/134 [==============================] - 17s 128ms/step - loss: 2.0685 - accuracy: 0.2307
Epoch 5/10
134/134 [==============================] - 17s 125ms/step - loss: 2.0482 - accuracy: 0.2311
Epoch 6/10
134/134 [==============================] - 17s 125ms/step - loss: 2.0401 - accuracy: 0.2309
Epoch 7/10
134/134 [==============================] - 17s 126ms/step - loss: 2.0315 - accuracy: 0.2485
Epoch 8/10
134/134 [==============================] - 17s 124ms/step - loss: 2.0128 - accuracy: 0.2440
Epoch 9/10
134/134 [==============================] - 16s 116ms/step - loss: 1.9853 - accuracy: 0.2658
Epoch 10/10
134/134 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - 2s 37ms/step


Processing Folds: 100%|██████████| 5/5 [4:29:30<00:00, 3234.07s/it]

Model training results saved to fold_model_performance_results.csv


In [4]:
%whos


Variable                       Type        Data/Info
----------------------------------------------------
Adam                           type        <class 'keras.src.optimizers.adam.Adam'>
Conv1D                         type        <class 'keras.src.layers.<...>olutional.conv1d.Conv1D'>
Dense                          type        <class 'keras.src.layers.core.dense.Dense'>
Dropout                        type        <class 'keras.src.layers.<...>ization.dropout.Dropout'>
Flatten                        type        <class 'keras.src.layers.<...>shaping.flatten.Flatten'>
LSTM                           type        <class 'keras.src.layers.rnn.lstm.LSTM'>
RandomForestClassifier         ABCMeta     <class 'sklearn.ensemble.<...>.RandomForestClassifier'>
Sequential                     type        <class 'keras.src.engine.sequential.Sequential'>
accuracy_score                 function    <function accuracy_score at 0x7fa9251d3010>
calculate_threshold_accuracy   function    <function calculate_t

In [5]:
%who_ls


['Adam',
 'Conv1D',
 'Dense',
 'Dropout',
 'Flatten',
 'LSTM',
 'RandomForestClassifier',
 'Sequential',
 'accuracy_score',
 'calculate_threshold_accuracy',
 'calculate_top3_accuracy',
 'classification_report',
 'data_dir',
 'evaluate_metrics',
 'f1_score',
 'load_data',
 'np',
 'os',
 'pd',
 'pickle',
 'precision_score',
 'recall_score',
 'root_dir',
 'tqdm',
 'train_all_models',
 'train_cnn',
 'train_lstm',
 'train_model_per_label',
 'train_random_forest']

In [6]:
# List all local variables
print(locals().keys())

# List all global variables
print(globals().keys())


dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__session__', '_i', '_ii', '_iii', '_i1', 'os', 'pickle', 'pd', 'np', 'RandomForestClassifier', 'accuracy_score', 'classification_report', 'f1_score', 'Sequential', 'Dense', 'Conv1D', 'Flatten', 'LSTM', 'Dropout', 'Adam', 'tqdm', 'calculate_top3_accuracy', 'calculate_threshold_accuracy', 'evaluate_metrics', 'load_data', 'train_random_forest', 'train_cnn', 'train_lstm', 'train_all_models', 'data_dir', 'root_dir', '_i2', 'precision_score', 'recall_score', 'train_model_per_label', '_i3', '_i4', '_i5', '_5', '_i6'])
dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__session__', '_i', '_ii', '_iii', '_i1', 'os', 'pickle', 'pd', 'np', 'RandomForestCla